In [20]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [21]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.service_account import Credentials

# Path to your JSON service account key file
SERVICE_ACCOUNT_FILE = '/content/software-engineer-assesments-0154df747a2c.json'

# Define the required scopes
SCOPES = ['https://www.googleapis.com/auth/documents']

# Authenticate using the service account
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize the Google Docs API with the authenticated credentials
docs_service = build('docs', 'v1', credentials=credentials)


[link text](https://)

In [22]:

# Step 1: Authenticate with Google Docs API
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import markdown

# Authenticate the user
auth.authenticate_user()

# Initialize the Google Docs API
docs_service = build('docs', 'v1')

# Step 2: Markdown content
markdown_content = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes.
"""

# Step 3: Parse markdown to structure content
def markdown_to_requests(md_content):
    lines = md_content.split("\n")
    requests = []
    index = 1

    for line in lines:
        if line.startswith("# "):  # Heading 1
            requests.append({"insertText": {"location": {"index": index}, "text": line[2:] + "\n"}})
            index += len(line[2:]) + 1
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": index - len(line[2:]) - 1, "endIndex": index - 1},
                    "textStyle": {"bold": True, "fontSize": {"magnitude": 24, "unit": "PT"}},
                    "fields": "bold,fontSize"
                }
            })
        elif line.startswith("## "):  # Heading 2
            requests.append({"insertText": {"location": {"index": index}, "text": line[3:] + "\n"}})
            index += len(line[3:]) + 1
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": index - len(line[3:]) - 1, "endIndex": index - 1},
                    "textStyle": {"bold": True, "fontSize": {"magnitude": 18, "unit": "PT"}},
                    "fields": "bold,fontSize"
                }
            })
        elif line.startswith("### "):  # Heading 3
            requests.append({"insertText": {"location": {"index": index}, "text": line[4:] + "\n"}})
            index += len(line[4:]) + 1
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": index - len(line[4:]) - 1, "endIndex": index - 1},
                    "textStyle": {"bold": True, "fontSize": {"magnitude": 16, "unit": "PT"}},
                    "fields": "bold,fontSize"
                }
            })
        elif line.startswith("- [ ]"):  # Checkboxes
            requests.append({"insertText": {"location": {"index": index}, "text": "☐ " + line[6:] + "\n"}})
            index += len("☐ " + line[6:]) + 1
        elif line.startswith("- "):  # Bullet points
            requests.append({"insertText": {"location": {"index": index}, "text": "• " + line[2:] + "\n"}})
            index += len("• " + line[2:]) + 1
        elif line.strip():  # Normal text
            requests.append({"insertText": {"location": {"index": index}, "text": line + "\n"}})
            index += len(line) + 1
        else:  # Blank lines
            requests.append({"insertText": {"location": {"index": index}, "text": "\n"}})
            index += 1

    return requests

# Convert the markdown content into Google Docs API requests
requests = markdown_to_requests(markdown_content)

# Step 4: Create and update the Google Doc
try:
    # Create a new Google Doc
    document = docs_service.documents().create(body={"title": "Product Team Sync"}).execute()
    document_id = document.get('documentId')

    # Update the document with the markdown content
    docs_service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()

    print(f"Document created successfully! View it here: https://docs.google.com/document/d/{document_id}/edit")

except HttpError as error:
    print(f"An error occurred: {error}")


Document created successfully! View it here: https://docs.google.com/document/d/19QxlduwoyvcE3-sZG1Wd-fxe3UJJ9OF5D4MPugPShPM/edit
